In [ ]:
import os
FLAG = os.getenv("FLAG", "SEKAI{TESTFLAG}")

## **1.Giải thích luồng chương trình**

### **1.1. Phương thức `play()`**
- `plainperm = bytes.fromhex(input(...))`: Chương trình yêu cầu nhập một **permutation** (một hoán vị của 0..255) ở dạng hex; `assert sorted(plainperm) == list(range(256))` kiểm tra rằng đó đúng là hoán vị (mỗi giá trị 0..255 xuất hiện đúng một lần) → 256 byte

-   `key = os.urandom(64)`: Sinh một `key` ngẫu nhiên 64 byte, chỉ server biết.
- *Phương thức `f(i)`*
    -   Với mỗi byte `k` trong `key[:-1]` (63 byte đầu), làm `i = plainperm[i ^ k]`.      
    -   Sau vòng lặp, trả `i ^ key[-1]` (xử lý XOR với byte cuối cùng).   
    -   Kết quả là `f` là một hoán vị trên không gian 0..255 (tổ hợp các XOR và tra hoán vị vẫn là hoán vị).
-   `cipherperm = bytes(map(f, range(256)))`: Chương trình áp dụng `f` cho từng giá trị 0..255, tạo ra một "cipher permutation" (hiển thị ở dạng hex). Nói cách khác, chương trình tiết lộ hoán vị `f` (kết quả của việc kết hợp plainperm và key).
-   Cuối cùng chương trình hỏi: "Do you know my key?": chờ bạn nhập key (hex). Nếu nhập đúng key (bytes), chương trình in FLAG; nếu sai thì vòng lặp cho chạy lại.

Hiểu đơn giản: Ta đưa vào một hoán vị `plainperm`, chương trình sinh `key` bí mật, tạo `cipherperm = f` (một hoán vị mới được in ra), rồi hỏi có biết `key` hay không. Nhiệm vụ của người tấn công là từ `plainperm` và `cipherperm` suy ra `key`.

In [ ]:
def play():
    plainperm = bytes.fromhex(input('Plainperm: '))
    assert sorted(plainperm) == list(range(256)), "Invalid permutation"

    key = os.urandom(64)
    def f(i):
        for k in key[:-1]:
            i = plainperm[i ^ k]            # i XOR k
        return i ^ key[-1]

    cipherperm = bytes(map(f, range(256)))  # Chuỗi byte kết quả
    print(f'Cipherperm: {cipherperm.hex()}')
    print('Do you know my key?')
    return bytes.fromhex(input()) == key

## **2. Toán học**
Ta có không gian $X=\{0,\dots,255\}$. Với mỗi byte $k$ định nghĩa phép toán trên $X$:

-   $XOR_k(x) = x \oplus k$ (một hoán vị vì phép XOR là đảo được, $a \oplus b \oplus b = a$),
    
-   $S(x) = \text{plainperm}[x]$.
    

Trong hàm `f`, mỗi bước `i = plainperm[i ^ k]` thực chất là áp dụng $S \circ XOR_k$. Với key là $[k_1,k_2,\dots,k_{64}]$ (một chuỗi 64 byte), ta có

$$
f = XOR_{k_{64}} \circ S \circ XOR_{k_{63}} \circ S \circ \cdots \circ XOR_{k_{1}}.
$$

(Chú ý thứ tự vì cách code làm: vòng lặp áp nhiều `S \circ XOR` rồi cuối cùng XOR với `k_last`.)

Vì các phép toán này đều là hoán vị, $f$ là một hoán vị biết được (vì chương trình in ra `cipherperm`). Người tấn công có hai hoán vị biết: $S$ (plainperm) và $f$ (cipherperm), còn bí mật là dãy các `XOR_k` xen kẽ với các $S$.


In [6]:
key = os.urandom(64)
print(list(key))
print(list(key[:-1]))

[119, 142, 236, 88, 38, 72, 139, 204, 234, 218, 121, 61, 195, 182, 92, 227, 32, 139, 192, 221, 24, 216, 113, 202, 96, 46, 81, 37, 127, 4, 11, 7, 1, 85, 8, 249, 86, 255, 38, 182, 242, 233, 221, 253, 29, 173, 1, 102, 103, 248, 240, 101, 123, 52, 171, 146, 211, 248, 246, 174, 148, 131, 113, 108]
[119, 142, 236, 88, 38, 72, 139, 204, 234, 218, 121, 61, 195, 182, 92, 227, 32, 139, 192, 221, 24, 216, 113, 202, 96, 46, 81, 37, 127, 4, 11, 7, 1, 85, 8, 249, 86, 255, 38, 182, 242, 233, 221, 253, 29, 173, 1, 102, 103, 248, 240, 101, 123, 52, 171, 146, 211, 248, 246, 174, 148, 131, 113]


In [1]:
print(24 ^ 8)

16
